In [65]:
import cv2 as cv
import sys
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [2]:
img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/antrenare/1_11.jpg"
# img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/imagini_auxiliare/02.jpg"
img1 = cv.imread(img_path)
img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/antrenare/1_12.jpg"
img2 = cv.imread(img_path)

In [3]:
def show_img(img,title):
    if img.shape[0] > 1000:
        img=cv.resize(img,(0,0),fx=0.20,fy=0.20)
    cv.imshow(title,img)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Use hsv colors to diferentiate the board from the rest of the background and get a mask

In [4]:
def crop_image_hsv(img):
    # img = cv.imread(img_path)
    # img=cv.resize(img,(0,0),fx=0.20,fy=0.20)
    # low_yellow = (14, 0, 0)
    # high_yellow = (255, 255, 255)
    # low_yellow = (0, 0, 0)
    # high_yellow = (14, 255, 255)
    low_yellow = (17, 0, 0)
    high_yellow = (116, 255, 255)
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)

    # kernel = np.ones((1,1),np.uint8)
    # mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_CLOSE, kernel)
    # cv.imshow("graay",mask_yellow_hsv)
    # image_bi_blur = cv.bilateralFilter(img, 15, 75, 75) 
    # gray = cv.cvtColor(image_bi_blur,cv.COLOR_BGR2GRAY)
    # print(mask_yellow_hsv.shape)
    # print(gray.shape)
    # mask_yellow_hsv = cv.bitwise_not(mask_yellow_hsv)
    # pre = cv.bitwise_and(gray,gray,mask=mask_yellow_hsv)
    # crop_img=cv.resize(crop_img,(0,0),fx=0.20,fy=0.20)
    # crop_img = extrage_careu(crop_img)
    # crop_img = cv.cvtColor(crop_img,cv.COLOR_BAYER_BG2GRAY)
    return mask_yellow_hsv

### Using the mask from the above function return only the board

In [5]:
def extrage_careu(original_img, cropped_mask_img):
    # image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    # image_m_blur = cv.medianBlur(image,3)
    # image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    # image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    # #show_image('image_sharpened',image_sharpened)
    # _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)
    # show_img(cropped_mask_img,'cropped_mask_img')
    kernel = np.ones((3, 4), np.uint8)
    cropped_mask_img = cv.erode(cropped_mask_img, kernel)
    
    # show_img(thresh,'image_thresholded')
    

    edges =  cv.Canny(cropped_mask_img,20,400)
    # cv.imshow("canny",edges)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2800
    height = 2800
    
    # image_copy = cv.cvtColor(original_img.copy(),cv.COLOR_GRAY2BGR)
    image_copy = original_img.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),1)
    # cv.imshow("detected corners",image_copy)
    # show_img(image_copy,"A")

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(original_img, M, (width, height))
    # result = cv.cvtColor(result,cv.COLOR_GRAY2BGR)
    
    return result

In [6]:
def preprocess(img):
    # image_bi_blur = cv.bilateralFilter(img, 15, 75, 75)

    # gray = cv.cvtColor(image_bi_blur,cv.COLOR_BGR2GRAY)

    # threshAdap = cv.adaptiveThreshold(gray, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 5, 2)
    # T, threshOtsu = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    # _, threshClassis1 = cv.threshold(crop_img1, 99, 255, cv.THRESH_BINARY_INV)
    # _, threshClassis1 = cv.threshold(gray, 95, 255, cv.THRESH_BINARY)
    
    return img

In [7]:
def get_boards_difference(resized_img1,resized_img2):
    crop_img1 = crop_image_hsv(resized_img1)
    careu_img1 = extrage_careu(original_img=resized_img1,cropped_mask_img=crop_img1)
    # preproc_careu1 = preprocess(careu_img1)

    crop_img2 = crop_image_hsv(resized_img2)
    careu_img2 = extrage_careu(original_img=resized_img2,cropped_mask_img=crop_img2)
    # preproc_careu2 = preprocess(careu_img2)

    # kernel = np.ones((3,3),np.uint8)
    result = cv.absdiff(careu_img2,careu_img1)

    # result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)

    return crop_img1,careu_img1,crop_img2,careu_img2,result
        

In [8]:
def generate_lines():
    lines_horizontal=[]
    for i in range(370,2452,148):
        l=[]
        l.append((350,i))
        l.append((2440,i))
        lines_horizontal.append(l)
        lines_vertical=[]
        
    for i in range(350,2600,150):
        l=[]
        l.append((i,370))
        l.append((i,2440))
        lines_vertical.append(l)

    return lines_horizontal,lines_vertical

In [9]:
def determina_configuratie_careu_ox(thresh,lines_horizontal,lines_vertical):
    # matrix = np.empty((9,9), dtype='str')
    max_i = 0
    max_j = 0
    max_medie = 0
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            Medie_patch=np.mean(patch)
            if Medie_patch > max_medie:
                max_i = i
                max_j = j
                max_medie = Medie_patch
    return max_i,max_j

In [10]:
# img1_r=cv.resize(img1,(0,0),fx=0.20,fy=0.20)
# img2_r=cv.resize(img2,(0,0),fx=0.20,fy=0.20)
img1_r=img1.copy()
img2_r=img2.copy()
crop_img1,careu_img1,crop_img2,careu_img2,result = get_boards_difference(img1_r,img2_r)

lines_horizontal,lines_vertical = generate_lines()

for line in  lines_vertical : 
    cv.line(careu_img2, line[0], line[1], (0, 0, 255), 5)
for line in  lines_horizontal : 
    cv.line(careu_img2, line[0], line[1], (0, 255, 0), 5)

# cv.imshow("preproc_careu1",preproc_careu1)
show_img(careu_img2,"preproc_careu1")
# show_img(careu_img1,"preproc_careu2")
# show_img(result,"result")
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (13,13))
result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)

i,j = determina_configuratie_careu_ox(result,lines_horizontal,lines_vertical)
print(i+1,end="")
print(chr(j+65))

# show_img(result,"result")
# show_img(careu_img2,"careu_img2")
# show_img(crop_img1,"crop_img1")
# cv.imwrite("careu.png",careu_img1)
# cv.imshow("careu_img1",preproc_careu2)
# cv.imshow("result",result)

cv.waitKey(0)
cv.destroyAllWindows()

6F


In [87]:
file = "../5_test.txt"
contents = Path(file).read_text()
a = "5_test.txt"
a[0]

'5'

In [66]:

i = 9
j = 8

contents = Path("../antrenare/1_26.txt").read_text()
contents[:-3].strip() == str(i+1) + str(chr(j+65))
contents[:-2].strip()

'10I'

In [ ]:
first_position = cv.imread("../imagini_auxiliare/01.jpg")
previous_img = first_position
training_path = '../antrenare/'
files=os.listdir(training_path)
lines_horizontal, lines_vertical = generate_lines()
game = 1
for file in files:
    if file[-3:]=='jpg':
        current_img = cv.imread(training_path + file)

        if str(game) != file[0]:
            game += 1
            previous_img = first_position

        crop_img1,careu_img1,crop_img2,careu_img2,result = get_boards_difference(previous_img,current_img)

        kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (13,13))
        result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)

        i,j = determina_configuratie_careu_ox(result,lines_horizontal,lines_vertical)
        
        print(i+1,end="")
        print(chr(j+65),end='\t')
        print(file)
    elif file[-3:] == 'txt' and file.find("scores.txt") ==  -1  and file.find("turns.txt") == -1:
        contents = Path(training_path + file).read_text()

        if contents[:-2].strip() != str(i+1) + str(chr(j+65)):
            print(contents[:-2])
            print(str(str(i+1) + chr(j+65)))
            show_img(result,"bad")

            break



    previous_img = current_img
        

9H	1_01.jpg
10H	1_02.jpg
11H	1_03.jpg
12H	1_04.jpg
6H	1_05.jpg
5H	1_06.jpg
4H	1_07.jpg
3H	1_08.jpg
2H	1_09.jpg
1H	1_10.jpg
6G	1_11.jpg
6F	1_12.jpg
6E	1_13.jpg
6I	1_14.jpg
6J	1_15.jpg
6K	1_16.jpg
6L	1_17.jpg
7I	1_18.jpg
8I	1_19.jpg
7J	1_20.jpg
5J	1_21.jpg
7K	1_22.jpg
7L	1_23.jpg
6D	1_24.jpg
9I	1_25.jpg
10I	1_26.jpg
10J	1_27.jpg
8J	1_28.jpg
8K	1_29.jpg
9K	1_30.jpg
5L	1_31.jpg
4L	1_32.jpg
6M	1_33.jpg
4J	1_34.jpg
8F	1_35.jpg
3J	1_36.jpg
2J	1_37.jpg
1J	1_38.jpg
11I	1_39.jpg
12I	1_40.jpg
13I	1_41.jpg
14I	1_42.jpg
9G	1_43.jpg
10G	1_44.jpg
9F	1_45.jpg
10F	1_46.jpg
10E	1_47.jpg
10D	1_48.jpg
9E	1_49.jpg
11E	1_50.jpg
7I	2_01.jpg
7J	2_02.jpg
7K	2_03.jpg
7L	2_04.jpg
7M	2_05.jpg
7N	2_06.jpg
7F	2_07.jpg
8I	2_08.jpg
9I	2_09.jpg
9H	2_10.jpg
9G	2_11.jpg
10G	2_12.jpg
10H	2_13.jpg
10I	2_14.jpg
10F	2_15.jpg
6H	2_16.jpg
6G	2_17.jpg
6F	2_18.jpg
8F	2_19.jpg
9F	2_20.jpg
9E	2_21.jpg
7E	2_22.jpg
6I	2_23.jpg
5H	2_24.jpg
4H	2_25.jpg
3H	2_26.jpg
2H	2_27.jpg
1H	2_28.jpg
6E	2_29.jpg
7D	2_30.jpg
7C	2_31.jpg
7B	2_32.jp